# Sea Surface Temperature indices of multidecadal variability

In [ ]:
import os
import sys
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot
matplotlib.rc_file('../rc_file')

In [ ]:
sys.path.append("..")
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon, make_map
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect
from filters import bandpass, highpass, chebychev
from regions import boolean_mask, global_ocean, SST_index_bounds
from plotting import shifted_color_map, discrete_cmap
from bd_analysis_indices import IndexAnalysis
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA
from xr_regression import xr_lintrend, xr_linear_trends_2D, xr_linear_trend, xr_quadtrend,\
                          ocn_field_regression, lag_linregress_3D

## indices from `SST_generation.py`

In [ ]:
ctrl_starts = np.arange(1, 152, 10)
lpd_starts = np.arange(154, 415, 10)

f, ax = plt.subplots(3, 1, figsize=(10,6))#, sharex=True)
for i, run in enumerate(['ctrl', 'lpd', 'had']):
    dt = [0, 200, 800][i]
    ax[2].set_xlabel('time [year]')
    for j, idx in enumerate(['AMO', 'SOM', 'TPI']):
        fn = f'{path_samoc}/SST/{idx}_{run}.nc'
        da = xr.open_dataarray(fn)
        if run=='had':
            label = 'two-factor'
            c = 'C2'
        else:
            label = 'scaled quadratic GMSST fit'
            c = 'C0'
        if i==0: label=None
        ax[j].plot(da[7:-7].time/365+dt, da[7:-7],         c=c, label=label)
        ax[j].plot(da[:7]  .time/365+dt, da[:7]  , ls=':', c=c)
        ax[j].plot(da[-7:] .time/365+dt, da[-7:] , ls=':', c=c)
            
        if run in ['ctrl', 'lpd']:
            for k, t in enumerate([ctrl_starts, lpd_starts][i]):
                tslice = (t, t+148)
                fns = f'{path_samoc}/SST/{idx}_{run}_{tslice[0]}_{tslice[1]}.nc'
                das = xr.open_dataarray(fns)
                if i==1 and k==0:  label='segment linear'
                else: label=None
                ax[j].plot(das[7:-7].time/365+dt, das[7:-7],         c='grey', lw=1, label=label)
                ax[j].plot(das[:7]  .time/365+dt, das[:7]  , ls=':', c='grey', lw=1)
                ax[j].plot(das[-7:] .time/365+dt, das[-7:] , ls=':', c='grey', lw=1)
            
            da = xr.open_dataarray(f'{path_samoc}/SST/{idx}_quadratic_pwdt_{run}.nc')
            
            if i==1: label = 'pointwise quadratic'
            ax[j].plot(da[7:-7].time/365+dt, da[7:-7],         c='C1', lw=1.5, label=label)
            ax[j].plot(da[:7]  .time/365+dt, da[:7]  , ls=':', c='C1', lw=1.5)
            ax[j].plot(da[-7:] .time/365+dt, da[-7:] , ls=':', c='C1', lw=1.5)
            
        ax[j].set_ylabel(idx)
        ax[j].axhline(0, c='k', lw=.5)
        if i>0 and j==1:
            ax[j].legend(ncol=4)
        ax[j].set_xticks(np.arange(0,1000,50))
ax[0].text(150, -.35, 'CTRL', ha='center')
ax[0].text(500, -.35, 'LPD' , ha='center')
ax[0].text(900, -.35, 'HAD' , ha='center')
ax[2].set_xlim((-20, 970))
plt.savefig(f'{path_results}/SST/SST_indices_different_dt')

In [ ]:
from bb_analysis_timeseries import AnalyzeTimeSeries as ATS

In [ ]:
f, ax = plt.subplots(3, 3, figsize=(10,6), sharex=True, sharey=True, constrained_layout=True)
for i, run in enumerate(['ctrl', 'lpd', 'had']):
    if run=='had':
        dt = 'GMST_tfdt'
    else:
        dt = 'quadratic_pwdt'
    
    for j, idx in enumerate(['AMO', 'SOM', 'TPI']):
        if idx=='TPI':
            da1 = xr.open_dataarray(f'{path_samoc}/SST/{idx}1_{dt}_raw_{run}.nc')
            da2 = xr.open_dataarray(f'{path_samoc}/SST/{idx}2_{dt}_raw_{run}.nc')
            da3 = xr.open_dataarray(f'{path_samoc}/SST/{idx}3_{dt}_raw_{run}.nc')
            da = da2 - (da1+da3)/2
        else:
            da = xr.open_dataarray(f'{path_samoc}/SST/{idx}_{dt}_raw_{run}.nc')
        
        ft, fc = 'lowpass', 13
        spec = ATS(da).spectrum(filter_type=ft, filter_cutoff=fc)  # spectrum
        rnsp = ATS(da).mc_ar1_spectrum(filter_type=ft, filter_cutoff=fc)  # red noise spectrum

        ax[j,i].plot(1/rnsp[1,:], rnsp[3,:]   , c='C1',                 label='AR(1) 95% C.I.')
        ax[j,i].plot(1/rnsp[1,:], rnsp[0,:]   , c='C1', lw=.5, ls=':' , label='AR(1) median')
        ax[j,i].plot(1/spec[1]  , spec[0]     , c='C0',                 label='MT spectrum')
        ax[j,i].loglog(1/spec[1], spec[2].T[1], c='C0', lw=.5, ls='--', label='jackknife est.')
        
        ax[j,0].set_ylabel(idx)
        ax[j,0].set_ylim((1e-4, 1e0))
        if i==0 and j==0:
            ax[j,i].legend(loc=4)
        ax[j,i].set_xlim((10,160))
        ax[j,i].set_xscale('log')
        ax[j,i].set_xticks([10,20,40,80,160])
        ax[j,i].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
        ax[j,i].set_yscale('log')
    
    ax[0,i].title.set_text(run.upper())
    ax[2,i].set_xlabel('period [years]')
plt.savefig(f'{path_results}/SST/SST_indices_spectra')

## generating the indices

In [ ]:
from SST_data_generation import DeriveSSTIndices

In [ ]:
%%time
# ca. 1:10 min for single ctrl run
# only seconds for lpd and had
for run in ['had', 'ctrl', 'lpd']:
    DeriveSSTIndices.derive_all_SST_avg_indices(run)

In [ ]:
# Filtering
# Lowpass filter
# cf. Chebychev filter results appended at bottom
for run in ['ctrl', 'lpd', 'had']:
    DeriveSSTIndices.derive_final_SST_indices(run)

In [ ]:
DeriveSSTIndices.derive_final_SST_indices('lpd')

In [ ]:
f, ax = plt.subplots(3,1, figsize=(8,8))
for i, run in enumerate(['had','ctrl', 'lpd']):
    for j, idx in enumerate(['AMO', 'SOM', 'TPI']):
        if run=='had':
            fns = [f'{path_samoc}/SST/{idx}_{run}.nc']
        elif run=='ctrl':
            fns = [f'{path_samoc}/SST/{idx}_100_248_{run}.nc', f'{path_samoc}/SST/{idx}_151_299_{run}.nc']
        elif run=='lpd':
            fns = [f'{path_samoc}/SST/{idx}_268_416_{run}.nc', f'{path_samoc}/SST/{idx}_417_565_{run}.nc']
        for fn in fns:
            da = xr.open_dataarray(fn, decode_times=False)
            ax[i].plot(np.arange(149), da.values, c=f'C{i}')

In [ ]:
AMO_GMST_had.plot()

In [ ]:
# SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc', decode_times=False)
# SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' , decode_times=False)
# SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' , decode_times=False)
# SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' , decode_times=False)
# SST_yrly_had  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_had.nc' , decode_times=False)

## analysisng the indices

In [ ]:
AMO = IndexAnalysis(index='AMO')
SOM = IndexAnalysis(index='SOM')
TPI = IndexAnalysis(index='TPI')

In [ ]:
%%time
for run in ['ctrl', 'lpd', 'had']:
    AMO.plot_yrly_regr_maps(run)
    SOM.plot_yrly_regr_maps(run)
    TPI.plot_yrly_regr_maps(run)
    

In [ ]:
%%time
# 7:15 for all
for Index in [AMO, SOM, TPI]:
    Index.make_regression_files()

In [ ]:
f, ax = plt.subplots(3, 3, figsize=(20,20))
for j, idx in enumerate(['AMO', 'SOM', 'TPI']):
    for i, run in enumerate(['had', 'ctrl', 'lpd']):
        da = xr.open_dataset(f'{path_samoc}/SST/{idx}_regr_{run}.nc').slope
        da.plot(ax=ax[i,j], vmax=0.5, vmin=-.5, cmap='RdBu_r')
plt.tight_layout()

# detrend with GMST (Kajtar et al. (2019))

In [ ]:
runs = ['ctrl', 'rcp', 'lpd', 'lpi', 'had']
indices = [AMO, TPI, SOM]

In [ ]:
for index in indices:
    index.plot_all_indices()

In [ ]:
%%time
for index in indices:
    index.plot_all_spectra()
    index.plot_all_autocorrelations()
    for run in runs:
        index.plot_spectrum_ar1(run)

### Variances

In [ ]:
def plot_variances(filter_type, cutoff):
    f = plt.figure(figsize=(5,5))
    plt.tick_params(labelsize=14)
    
    f = bandpass
    plt.title(f'{filter_type}: {cutoff[1]}-{cutoff[0]} years', fontsize=14)
    fn = f'{path_results}/SST/SST_index_variance_{cutoff[1]}_{cutoff[0]}_years'
        
    Ms = []
    for i, index in enumerate(indices):
        for j, run in enumerate(runs):
            m = ['D','s','^','v','o'][j]
            M = plt.scatter(i, f(index.all_indices[run], cutoff).std(dim='time'), c=f'C{j}', s=80, marker=m, label=run.upper())
            if i==0: Ms.append(M)
#     plt.legend(fontsize=14, loc=1)
    plt.legend(handles=Ms, fontsize=14, loc=9)
    plt.xticks(range(3), ['AMO', 'TPI', 'SOM'], fontsize=14)
    plt.ylabel('standard deviation [K]', fontsize=14)
    plt.tight_layout()
    plt.savefig(fn)
    
plot_variances('bandpass', np.array([30,10]))
plot_variances('bandpass', np.array([50,20]))
plot_variances('bandpass', np.array([100,30]))

### Regression maps 

In [ ]:
ds = xr.open_dataset(f'{path_samoc}/SST/AMO_regr_lpd.nc')

In [ ]:
%%time
for Index in [AMO, SOM, TPI]:
    Index.plot_regression_map('had')
    Index.plot_regression_map('ctrl')
    Index.plot_regression_map('lpd')

In [ ]:
type(rect_polygon(SST_index_bounds('AMO')))

In [ ]:
# %%time
# # 2:55
# AMO_GMST_ctrl = SST_index('AMO', 'ctrl', detrend_signal='GMST')
# AMO_GMST_rcp  = SST_index('AMO', 'rcp' , detrend_signal='GMST')
# AMO_GMST_lpi  = SST_index('AMO', 'lpi' , detrend_signal='GMST')
# AMO_GMST_lpd  = SST_index('AMO', 'lpd' , detrend_signal='GMST')
# AMO_GMST_had  = SST_index('AMO', 'had' , detrend_signal='GMST')

# AMO_rgnl_ctrl = SST_index('AMO', 'ctrl', detrend_signal='AMO')
# AMO_rgnl_rcp  = SST_index('AMO', 'rcp' , detrend_signal='AMO')
# AMO_rgnl_lpi  = SST_index('AMO', 'lpi' , detrend_signal='AMO')
# AMO_rgnl_lpd  = SST_index('AMO', 'lpd' , detrend_signal='AMO')
# AMO_rgnl_had  = SST_index('AMO', 'had' , detrend_signal='AMO')

In [ ]:
AMO_GMST_ctrl = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_ctrl.nc')
AMO_GMST_rcp  = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_rcp.nc' )
AMO_GMST_lpi  = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_lpi.nc' )
AMO_GMST_lpd  = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_lpd.nc' )
AMO_GMST_had  = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_had.nc' )

AMO_rgnl_ctrl = xr.open_dataarray(f'{path_samoc}/SST/AMO_raw_ctrl.nc')
AMO_rgnl_rcp  = xr.open_dataarray(f'{path_samoc}/SST/AMO_raw_rcp.nc' )
AMO_rgnl_lpi  = xr.open_dataarray(f'{path_samoc}/SST/AMO_raw_lpi.nc' )
AMO_rgnl_lpd  = xr.open_dataarray(f'{path_samoc}/SST/AMO_raw_lpd.nc' )
AMO_rgnl_had  = xr.open_dataarray(f'{path_samoc}/SST/AMO_AMO_dt_raw_had.nc' )

AMO_rgnl_ctrl -= xr_lintrend(AMO_rgnl_ctrl)
AMO_rgnl_rcp  -= xr_quadtrend(AMO_rgnl_rcp )
AMO_rgnl_lpi  -= xr_lintrend(AMO_rgnl_lpi )
AMO_rgnl_lpd  -= xr_lintrend(AMO_rgnl_lpd )

In [ ]:
AMO_ctrl = lowpass(AMO_rgnl_ctrl, 13) 
AMO_rcp  = lowpass(AMO_rgnl_rcp , 13) 
AMO_lpi  = lowpass(AMO_rgnl_lpi , 13) 
AMO_lpd  = lowpass(AMO_rgnl_lpd , 13) 
AMO_had  = lowpass(AMO_rgnl_had , 13)

AMO_ctrl.to_netcdf(f'{path_samoc}/SST/AMO_ctrl.nc')
AMO_rcp .to_netcdf(f'{path_samoc}/SST/AMO_rcp.nc' )
AMO_lpi .to_netcdf(f'{path_samoc}/SST/AMO_lpi.nc' )
AMO_lpd .to_netcdf(f'{path_samoc}/SST/AMO_lpd.nc' )
AMO_had .to_netcdf(f'{path_samoc}/SST/AMO_had.nc' )

In [ ]:
AMO_GMST_dt_ctrl = lowpass(AMO_GMST_ctrl, 13) 
AMO_GMST_dt_rcp  = lowpass(AMO_GMST_rcp , 13) 
AMO_GMST_dt_lpi  = lowpass(AMO_GMST_lpi , 13) 
AMO_GMST_dt_lpd  = lowpass(AMO_GMST_lpd , 13) 
AMO_GMST_dt_had  = lowpass(AMO_GMST_had , 13)

AMO_GMST_dt_ctrl.to_netcdf(f'{path_samoc}/SST/AMO_GMST_dt_ctrl.nc')
AMO_GMST_dt_rcp .to_netcdf(f'{path_samoc}/SST/AMO_GMST_dt_rcp.nc' )
AMO_GMST_dt_lpi .to_netcdf(f'{path_samoc}/SST/AMO_GMST_dt_lpi.nc' )
AMO_GMST_dt_lpd .to_netcdf(f'{path_samoc}/SST/AMO_GMST_dt_lpd.nc' )
AMO_GMST_dt_had .to_netcdf(f'{path_samoc}/SST/AMO_GMST_dt_had.nc' )

In [ ]:
AMO_rgnl_rcp.plot()

In [ ]:
GMSTs = [AMO_GMST_ctrl, AMO_GMST_rcp, AMO_GMST_lpi, AMO_GMST_lpd, AMO_GMST_had ]
rgnls = [AMO_rgnl_ctrl, AMO_rgnl_rcp, AMO_rgnl_lpi, AMO_rgnl_lpd, AMO_rgnl_had ]

f, ax = plt.subplots(5, 1, figsize=(8,12))
for i in range(5):
    r = np.corrcoef(xr.align(lowpass(GMSTs[i], 13), lowpass(rgnls[i], 13)))[0,1]
    run = ['ctrl', 'rcp', 'lpd', 'lpi', 'had'][i]
    ax[i].set_ylabel(f'AMO index [K]', fontsize=14)
    ax[i].text(.02, .85, run.upper(), transform=ax[i].transAxes, fontsize=16)
    ax[i].text(.98, .85, f'r={r:4.2f}', transform=ax[i].transAxes, fontsize=12, horizontalalignment='right')
    ax[i].tick_params(labelsize=14)
    ax[i].axhline(0, c='k', lw=.5)
   
    rgnls[i] -= rgnls[i].mean(dim='time')
    if i==4:
        time_G = GMSTs[i].time/365 + 1870
        time_r = rgnls[i].time/365 + 1870
    else: 
        time_G = GMSTs[i].time/365
        time_r = rgnls[i].time/365
        
    ax[i].plot(time_G,         GMSTs[i]     , label=r'raw AMO$_{\mathrm{GMST}}$', alpha=.3)
    ax[i].plot(time_G, lowpass(GMSTs[i], 13), label=r'AMO$_{\mathrm{GMST}}$')
        
    ax[i].plot(time_r,         rgnls[i]     , label=r'raw AMO$_{\mathrm{rgnl}}$', alpha=.3)
    ax[i].plot(time_r, lowpass(rgnls[i], 13), label=r'AMO$_{\mathrm{rgnl}}$')
    if i==0:
        ax[i].legend(loc=4, ncol=4, fontsize=12)
ax[4].set_xlabel('time [years]', fontsize=14)
f.align_ylabels()
plt.tight_layout()
plt.savefig(f'{path_results}/SST/AMO_overview_GMST_rgnl')

I decide to use the 13 year lowpass filtered `rgnl` AMO signal, because:
1. the whole SST time series is available, because I detrend with SSTs
1. it is very similar to the scaled GMST time series

# SST indices

In [ ]:
MASK_ocn  = boolean_mask('ocn'     , 0)
MASK_low  = boolean_mask('ocn_low' , 0)
MASK_rect = boolean_mask('ocn_rect', 0)

In [ ]:
TAREA       = xr_AREA('ocn')
AREA_low    = xr_AREA('ocn_low')
AREA_rect   = xr_AREA('ocn_rect')

# Atlantic Ocean

## Atlantic Multidecadal Oscillation
from Wikipedia:
> Several methods have been proposed to remove the global trend and El Niño-Southern Oscillation (ENSO) influence over the North Atlantic SST. Trenberth and Shea, assuming that the effect of global forcing over the North Atlantic is similar to the global ocean, subtracted the global (60°N-60°S) mean SST from the North Atlantic SST to derive a revised AMO index.[6]
>
> Ting et al. however argue that the forced SST pattern is not globally uniform; they separated the forced and internally generated variability using signal to noise maximizing EOF analysis.[2]
>
> Van Oldenborgh et al. derived an AMO index as the SST averaged over the extra-tropical North Atlantic (to remove the influence of ENSO that is greater at tropical latitude) minus the regression on global mean temperature.[7]
>
> Guan and Nigam removed the non stationary global trend and Pacific natural variability before applying an EOF analysis to the residual North Atlantic SST.[8]
>
> The linearly detrended index suggests that the North Atlantic SST anomaly at the end of the twentieth century is equally divided between the externally forced component and internally generated variability, and that the current peak is similar to middle twentieth century; by contrast the others methodology suggest that a large portion of the North Atlantic anomaly at the end of the twentieth century is externally forced.[2]

We use the method of Trenberth here: averaged SST [0-60 N]x[0-80W] - avg. SST [60S-60N], then .

In [ ]:
# plotting tropics-extratropics and North atlantic maps
f,ax = plt.subplots(2,2, figsize=(8,5), sharey='row')
for i, domain in enumerate(['ocn', 'ocn_low']):
    xr_AREA(domain).where(TexT_mask(domain)).plot(ax=ax[i,0])
    xr_AREA(domain).where(AMO_mask(domain)).plot(ax=ax[i,1])
plt.tight_layout()

In [ ]:
# without detrending

def AMO(run):
    if run in ['ctrl', 'rcp']:
        domain    = 'ocn'
        AREA      = TAREA
        MASK      = MASK_ocn
    elif run in ['lpi', 'lpd']:
        domain    = 'ocn_low'
        AREA      = AREA_low
        MASK      = MASK_low
        
    if run =='ctrl':   SST_yrly = SST_yrly_ctrl
    elif run =='rcp':  SST_yrly = SST_yrly_rcp
    elif run =='lpd':  SST_yrly = SST_yrly_lpd
    elif run =='lpi':  SST_yrly = SST_yrly_lpi
        
    dims = ('nlat', 'nlon')
    AREA_AMO  = AREA.where(AMO_mask(domain)).sum()
    AREA_TexT = AREA.where(TexT_mask(domain)).sum()
    print(AREA_AMO.values, AREA_TexT.values)
    
    NA   = SST_index(xa_SST=SST_yrly, AREA=AREA, index_loc=None, AREA_index=AREA_AMO , MASK=AMO_mask(domain) , dims=dims)
    TexT = SST_index(xa_SST=SST_yrly, AREA=AREA, index_loc=None, AREA_index=AREA_TexT, MASK=TexT_mask(domain), dims=dims)
    
    AMO  = (NA - TexT) - (NA - TexT).mean()
    AMO.to_netcdf(f'{path_results}/SST/AMO_yrly_{run}.nc')

    f,ax = plt.subplots(3,1,figsize=(8,5),sharex=True)
    ax[0].plot(NA)
    ax[1].plot(TexT)
    ax[2].plot(AMO)
    plt.tight_layout()
    return AMO

In [ ]:
# %%time
# AMO_ctrl = AMO('ctrl')  # 1:20
# AMO_rcp  = AMO('rcp' )  # 40 sec
# AMO_lpd  = AMO('lpd' )  # 4 sec
# AMO_lpi  = AMO('lpi' )  # 9 sec

In [ ]:
AMO_ctrl = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_ctrl.nc', decode_times=False)
AMO_rcp  = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_rcp.nc' , decode_times=False)
AMO_lpd  = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_lpd.nc' , decode_times=False)
AMO_lpi  = xr.open_dataarray(f'{path_results}/SST/AMO_yrly_lpi.nc' , decode_times=False)

In [ ]:
# comparing filtering methods and timescales
plt.figure(figsize=(8,5))
plt.axhline(0,c='k', lw=.5)
plt.plot(AMO_ctrl)
plt.plot(AMO_ctrl.rolling(time=10, center=True).mean())
plt.plot(AMO_ctrl.rolling(time=40, center=True).mean())
plt.plot(lowpass(AMO_ctrl.values, 40))
plt.plot(lowpass(AMO_ctrl.values, 10))

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
ax.axhline(0,c='k', lw=.5)
ax.plot([2050]*2, [-.15,.2], c='g', lw=.5)
ax.plot([2200]*2, [-.15,.2], c='g', lw=.5)

L1, = ax.plot(AMO_ctrl.time/365+1850, AMO_ctrl, c='C0', ls='-', lw=1, label='lowpass 13 yr')
ax.plot(      AMO_rcp .time/365+ 200, AMO_rcp , c='C1', ls='-', lw=1)
ax.plot(      AMO_lpd .time/365+1350, AMO_lpd , c='C2', ls='-', lw=1)
ax.plot(      AMO_lpi .time/365-1600, AMO_lpi , c='C3', ls='-', lw=1)
ax.plot(      AMO_had .time/365+2350, AMO_had , c='C4', ls='-', lw=1)

L2, = ax.plot(AMO_ctrl.time/365+1850, lowpass(AMO_ctrl, 40), c='C0', ls='--' , lw=1.5, label='low pass 40 yr')
ax.plot(      AMO_rcp .time/365+ 200, lowpass(AMO_rcp , 40), c='C1', ls='--' , lw=1.5)
ax.plot(      AMO_lpd .time/365+1350, lowpass(AMO_lpd , 40), c='C2', ls='--' , lw=1.5)
ax.plot(      AMO_lpi .time/365-1600, lowpass(AMO_lpi , 40), c='C3', ls='--' , lw=1.5)
ax.plot(      AMO_had .time/365+2350, lowpass(AMO_had , 40), c='C4', ls='--' , lw=1.5)

L3, = ax.plot(AMO_ctrl.time/365+1850, xr_lintrend(AMO_ctrl), c='C0', ls='-.', lw=1.5, label='linear trend')
ax.plot(      AMO_rcp .time/365+ 200, xr_lintrend(AMO_rcp ), c='C1', ls='-.', lw=1.5)
ax.plot(      AMO_lpd .time/365+1350, xr_lintrend(AMO_lpd ), c='C2', ls='-.', lw=1.5)
ax.plot(      AMO_lpi .time/365-1600, xr_lintrend(AMO_lpi ), c='C3', ls='-.', lw=1.5)
ax.plot(      AMO_had .time/365+2350, xr_lintrend(AMO_had ), c='C4', ls='-.', lw=1.5)

ax.text(1950, .21, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, .21, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, .21, 'pres. day low', fontsize=16, color='C2')
ax.text(1270, .21, 'pre-ind. low' , fontsize=16, color='C3')
ax.text(2350, .21, 'HadISST'      , fontsize=16, color='C4')

ax.legend(handles=[L1, L2, L3], loc=8, ncol=3, fontsize=14, frameon=False)
ax.set_xlabel('time[years]', fontsize=16)
ax.set_ylabel('AMO index [$^\circ$C]'  , fontsize=16)
ax.set_ylim((-.24,.24))
ax.set_xticks(np.arange(1200,2400,200))
f.tight_layout()
# plt.savefig(f'{path_results}/SST/AMO_index_overview')

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
ax.axhline(0,c='k', lw=.5)
ax.plot([2050]*2, [-.15,.2], c='g', lw=.5)
ax.plot([2200]*2, [-.15,.2], c='g', lw=.5)

L1, = ax.plot(AMO_ctrl.time/365+1850, chebychev(AMO_ctrl.values, 13), c='C0', ls='--', lw=1, label='chebychev 13 yr')
ax.plot(      AMO_rcp .time/365+ 200, chebychev(AMO_rcp .values, 13), c='C1', ls='--', lw=1)
ax.plot(      AMO_lpd .time/365+1350, chebychev(AMO_lpd .values, 13), c='C2', ls='--', lw=1)
ax.plot(      AMO_lpi .time/365-1600, chebychev(AMO_lpi .values, 13), c='C3', ls='--', lw=1)

L2, = ax.plot(AMO_ctrl.time/365+1850, lowpass(AMO_ctrl.values, 40), c='C0', ls='-' , lw=2, label='low pass 40 yr')
ax.plot(      AMO_rcp .time/365+ 200, lowpass(AMO_rcp .values, 40), c='C1', ls='-' , lw=2)
ax.plot(      AMO_lpd .time/365+1350, lowpass(AMO_lpd .values, 40), c='C2', ls='-' , lw=2)
ax.plot(      AMO_lpi .time/365-1600, lowpass(AMO_lpi .values, 40), c='C3', ls='-' , lw=2)

L3, = ax.plot(AMO_ctrl.time/365+1850, xr_lintrend(chebychev(AMO_ctrl,13)), c='C0', lw=1.5, ls='-.', label='linear trend')
ax.plot(      AMO_rcp .time/365+ 200, xr_lintrend(chebychev(AMO_rcp ,13)), c='C1', lw=1.5, ls='-.')
ax.plot(      AMO_lpd .time/365+1350, xr_lintrend(chebychev(AMO_lpd ,13)), c='C2', ls='-.' , lw=1.5)
ax.plot(      AMO_lpi .time/365-1600, xr_lintrend(chebychev(AMO_lpi ,13)), c='C3', ls='-.' , lw=1.5)

ax.text(1950, .21, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, .21, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, .21, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, .21, 'pre-ind. low' , fontsize=16, color='C3')

ax.legend(handles=[L1, L2, L3], loc=8, ncol=3, fontsize=14, frameon=False)
ax.set_xlabel('time[years]', fontsize=16)
ax.set_ylabel('AMO index [$^\circ$C]'  , fontsize=16)
ax.set_ylim((-.17,.24))
ax.set_xticks(np.arange(0,2400,200))
f.tight_layout()
# plt.savefig(f'{path_results}/SST/AMO_index_overview')

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
ax.axhline(0,c='k', lw=.5)
ax.axvline(200,c='g', lw=.5)
ax.axvline(300,c='g', lw=.5)
L1, = ax.plot(AMO_ctrl.time/365     , AMO_ctrl, c='C0', lw=.5, ls=':', label='yrly raw data')
ax.plot(AMO_rcp .time/365-1700, AMO_rcp , c='C1', lw=.5, ls=':')
# ax.plot(AMO_ctrl.time/365     , AMO_ctrl.rolling(time=15, center=True).mean(), c='C0', ls='-.', lw=1.5, label='15 yr running mean')
# ax.plot(AMO_rcp .time/365-1700, AMO_rcp .rolling(time=15, center=True).mean(), c='C1', ls='-.', lw=1.5)
L2, = ax.plot(AMO_ctrl.time/365     , lowpass(AMO_ctrl.values, 15), c='C0', lw=1.5, ls='--', label='low pass 15 yr')
ax.plot(AMO_rcp .time/365-1700, lowpass(AMO_rcp .values, 15), c='C1', lw=1.5, ls='--')
L3, = ax.plot(AMO_ctrl.time/365     , lowpass(AMO_ctrl.values, 40), c='C0', lw=2, label='low pass 40 yr')
ax.plot(AMO_rcp .time/365-1700, lowpass(AMO_rcp .values, 40), c='C1', lw=2)
L4, = ax.plot(AMO_ctrl.time/365     , xr_lintrend(AMO_ctrl), c='C0', lw=1.5, ls='-.', label='linear trend')
ax.plot(AMO_rcp .time/365-1700, xr_lintrend(AMO_rcp ), c='C1', lw=1.5, ls='-.')

ax.text(.05, .92, 'CTRL', transform=ax.transAxes, fontsize=16, color='C0')
ax.text(.72, .92, 'RCP', transform=ax.transAxes, fontsize=16, color='C1')
ax.legend(handles=[L1, L2, L3, L4], loc=3, ncol=4, fontsize=14, frameon=False)
ax.set_xlabel('time [years]' , fontsize=16)
ax.set_ylabel('AMO index [K]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/SST/AMO_index_ctrl_rcp')

In [ ]:
# filtered and detrended AMO indices
AMO_filt_detr_ctrl = chebychev(AMO_ctrl, 13)-xr_lintrend(chebychev(AMO_ctrl,13))
AMO_filt_detr_rcp  = chebychev(AMO_rcp , 13)-xr_lintrend(chebychev(AMO_rcp ,13))
AMO_filt_detr_lpd  = chebychev(AMO_lpd , 13)-xr_lintrend(chebychev(AMO_lpd ,13))
AMO_filt_detr_lpi  = chebychev(AMO_lpi , 13)-xr_lintrend(chebychev(AMO_lpi ,13))
AMO_filt_detr_ctrl.to_netcdf(f'{path_results}/SST/AMO_filt_detr_ctrl.nc')
AMO_filt_detr_rcp .to_netcdf(f'{path_results}/SST/AMO_filt_detr_rcp.nc' )
AMO_filt_detr_lpd .to_netcdf(f'{path_results}/SST/AMO_filt_detr_lpd.nc' )
AMO_filt_detr_lpi .to_netcdf(f'{path_results}/SST/AMO_filt_detr_lpi.nc' )

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
ax.axhline(0,c='k', lw=.5)
ax.plot([2050]*2, [-.15,.13], c='g', lw=.5)
ax.plot([2200]*2, [-.15,.13], c='g', lw=.5)

L1, = ax.plot(AMO_ctrl.time/365+1850, AMO_filt_detr_ctrl, c='C0', ls='--', lw=1, label='chebychev 13 yr')
ax.plot(AMO_rcp .time/365+ 200,       AMO_filt_detr_rcp , c='C1', ls='--', lw=1)
ax.plot(AMO_lpd .time/365+1350,       AMO_filt_detr_lpd , c='C2', ls='--', lw=1)
ax.plot(AMO_lpi .time/365-1600,       AMO_filt_detr_lpi , c='C3', ls='--', lw=1)

L2, = ax.plot(AMO_ctrl.time/365+1850, lowpass(AMO_ctrl.values, 40)-xr_lintrend(chebychev(AMO_ctrl,13)), c='C0', ls='-' , lw=1.5, label='low pass 40 yr')
ax.plot(AMO_rcp .time/365+ 200,       lowpass(AMO_rcp .values, 40)-xr_lintrend(chebychev(AMO_rcp ,13)), c='C1', ls='-' , lw=1.5)
ax.plot(AMO_lpd .time/365+1350,       lowpass(AMO_lpd .values, 40)-xr_lintrend(chebychev(AMO_lpd ,13)), c='C2', ls='-' , lw=1.5)
ax.plot(AMO_lpi .time/365-1600,       lowpass(AMO_lpi .values, 40)-xr_lintrend(chebychev(AMO_lpi ,13)), c='C3', ls='-' , lw=1.5)

ax.text(1950, .15, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, .15, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, .15, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, .15, 'pre-ind. low' , fontsize=16, color='C3')

ax.legend(handles=[L1, L2], loc=8, ncol=3, fontsize=14, frameon=False)
ax.set_xlabel('time[years]', fontsize=16)
ax.set_ylabel('detrended AMO index [$^\circ$C]'  , fontsize=16)
ax.set_ylim((-.17,.18))
ax.set_xticks(np.arange(0,2400,200))
f.tight_layout()
plt.savefig(f'{path_results}/SST/AMO_detrended_overview')

- comparable period to observations (approx 70 years; Trenberth & Shea)
- much smaller amplitude though (0.05K vs. 0.2K)

In [ ]:
# periodogram
AMOs = [AMO_ctrl, AMO_rcp, AMO_lpd, AMO_lpi]

f, ax = plt.subplots(1, 1, figsize=(8,5), sharey=True)
ax.set_ylabel('spectral power', fontsize=16)

for i in range (4):
    label= ['CTRL', 'RCP', 'LPD', 'LPI'][i]
    f, Pxx = sp.signal.welch(AMOs[i]-xr_lintrend(AMOs[i]), fs=1)
    ax.loglog(1/f, np.sqrt(Pxx), label=label)

ax.tick_params(labelsize=14)
ax.set_xlabel('period [yr]', fontsize=16)
ax.legend(fontsize=14, frameon=False)
    
plt.tight_layout()
# plt.savefig(f'{path_results}/SST/ENSO_spectrum')

In [ ]:
SST_yrly_detr_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_ctrl.nc', decode_times=False)

In [ ]:
SST_yrly_detr_ctrl[:,2000,1000].plot()
(10*AMO_filt_detr_ctrl).plot()

In [ ]:
# Chebychev filter
# for run in ['ctrl', 'lpd', 'had']:
#     DeriveSSTIndices.derive_final_SST_indices(run)